In [72]:
import matplotlib as mpl
%matplotlib inline

In [6]:
import pandas as pd
import io
import requests
import os
from prefect import Flow, task

----------
ETL

----------

In [10]:
@task
def extract(local=True):
    
    if local:
        df = pd.read_csv("../data/chiffres-cles.csv")
        quatre_taux = pd.read_csv("../data/table-indicateurs-open-data-dep-serie.csv")
    else: 
        url = "https://github.com/opencovid19-fr/data/raw/master/dist/chiffres-cles.csv"
        s = requests.get(url).content
        df = pd.read_csv(io.StringIO(s.decode('utf-8')))
        
        url= "https://www.data.gouv.fr/fr/datasets/r/4acad602-d8b1-4516-bc71-7d5574d5f33e"
        s=requests.get(url).content
        quatre_taux = pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    dpt = pd.read_csv("../data/departements-region.csv")
    reg = pd.read_csv("../data/regions-france.csv")
    pop = pd.read_csv("../data/estim_pop.csv")

        
    df["date"] = df["date"].replace("_", "-", regex=True)
    df["date"] = pd.to_datetime(df["date"])

    df["dep_code"] = df[(df['maille_code'].str.startswith('DEP')) | (df['maille_code'].str.startswith('COM'))]["maille_code"]\
    .replace(".*-", "", regex=True)
    df["reg_code"] = df[df['maille_code'].str.startswith('REG')]["maille_code"]\
    .replace(".*-", "", regex=True).astype("int64")

    quatre_taux["extract_date"] = pd.to_datetime(quatre_taux["extract_date"])


    geo = dpt.merge(reg, right_on= 'nom_region', left_on='region_name')\
    .rename(columns = {'num_dep':'code_dep'})\
    .drop(columns=["nom_region"])[['code_region', 'region_name', 'code_dep', 'dep_name']]

    pop[["2020", "2021"]] = pop[["2020", "2021"]].replace("\s", "", regex=True).astype(int)
    
    return [df, quatre_taux, geo, pop]

https://www.data.gouv.fr/fr/datasets/indicateurs-de-suivi-de-lepidemie-de-covid-19/
https://www.data.gouv.fr/fr/datasets/donnees-relatives-aux-resultats-des-tests-virologiques-covid-19/

In [9]:
@task
def transform(dataframe):
    
    df, quatre_taux, geo, pop = dataframe[0], dataframe[1], dataframe[2], dataframe[3]
    
    df_clean = df.merge(geo, right_on="code_dep", left_on='dep_code')[['date', 'code_region', 'region_name', 'code_dep',
           'dep_name', 'cas_confirmes',
           'cas_ehpad', 'cas_confirmes_ehpad', 'cas_possibles_ehpad', 'deces',
           'deces_ehpad', 'reanimation', 'hospitalises',
           'nouvelles_hospitalisations', 'nouvelles_reanimations', 'gueris',
           'depistes', 'source_nom', 'source_url', 'source_archive', 'source_type']]

    rs = []
    for ind, row in df_clean.iterrows():   #[["deces","gueris", 'reanimation', 'hospitalises']]
        if row["date"].year == 2020:
            effectif = pop.loc[pop['code_dpt'] == row['code_dep'], "2020"]
        else: 
            effectif = pop.loc[pop['code_dpt'] == row['code_dep'], "2021"]
        dc = row["deces"] / effectif * 100000
        gu = row["gueris"] / effectif * 100000
    #     rea = row["reanimation"] / effectif * 100000
        hos = row["hospitalises"] / effectif * 100000
    #     tot= dc.values[0] + gu.values[0] + hos.values[0]

        rs.append([row["date"], row["region_name"], row["code_dep"], row["dep_name"], dc.values[0], gu.values[0], hos.values[0]])
    col = ['date', 'region_name', 'dep_code', 'dep_name', 'deces', 'gueris', 'symptomatiques']            


    final = pd.DataFrame(rs, columns = col)\
    .merge(quatre_taux[["extract_date",'departement', 'tx_incid']], left_on=['date','dep_code'], right_on=["extract_date",'departement'])\
    .drop(columns=["extract_date",'departement'])#\
#     .rename(columns= {"tx_incid" : "susceptibles"})

    final["asymptomatiques"] = final["tx_incid"] - final['symptomatiques']

    start_date = final[final['asymptomatiques']>0].sort_values('date').iloc[0,0]

    final = final.loc[(final['date'] >= start_date)  , : ]
    
    return final

In [ ]:
@task
def load(file, path = "../data/df_clean.csv"):
    if path = "../data/df_clean.csv":
        os.rename("../data/df_clean.csv", "../data/_old_df_clean.csv")
    file.to_csv(path

In [134]:
final.to_csv("../data/df_clean.csv")

In [ ]:
# prefect
schedule = Schedule(
  clocks=[IntervalClock(timedelta(hours=24))]
#     ,
#   filters=[is_weekday],
#   or_filters=[
#     between_times(pendulum.time(9), pendulum.time(9)),
#     between_times(pendulum.time(15), pendulum.time(15)),
#   ],
#   not_filters=[between_dates(1, 1, 1, 31)],
#   adjustments=[next_weekday]
)

with Flow("Schedule covid", schedule=schedule) as flow:
    e = extract(local=False)
    t = transform(e)
    l = load(t)
try:
    flow.run()
except:
    e = extract(local= True)
    t = transform(e)
    l = load(t)
    

In [152]:
pop.to_csv("../data/pop.csv")

-------
SEAIR

-------

In [151]:
final 

,date,region_name,dep_code,dep_name,deces,gueris,hospitalises,total,susceptibles,asymptomatiques
3905,2020-05-14,Nouvelle-Aquitaine,79,Deux-Sèvres,5.071969,14.415069,2.669457,22.156494,2.95,0.280543
38236,2020-05-14,Occitanie,11,Aude,14.625365,52.651313,5.318314,72.594992,12.61,7.291686
38237,2020-05-15,Occitanie,11,Aude,14.625365,53.449060,4.520567,72.594992,12.88,8.359433
23658,2020-05-15,Occitanie,12,Aveyron,8.233871,44.391302,7.159887,59.785060,9.70,2.540113
4333,2020-05-15,Nouvelle-Aquitaine,86,Vienne,8.439974,27.144780,3.877826,39.462579,5.72,1.842174
...,...,...,...,...,...,...,...,...,...,...
37310,2021-05-11,Occitanie,82,Tarn-et-Garonne,78.931113,316.868380,25.929061,421.728553,104.71,78.780939
13307,2021-05-11,Bretagne,56,Morbihan,53.334871,245.183154,24.112079,322.630104,128.78,104.667921
8586,2021-05-11,Hauts-de-France,80,Somme,156.194358,506.838351,56.060729,719.093438,200.26,144.199271
28755,2021-05-11,Corse,2A,Corse-du-Sud,70.044792,290.010015,27.034832,387.089639,40.02,12.985168


In [145]:
#SEAIRD
import numpy as np

def params(data):
    alpha = data.iloc[0, -4] / data.iloc[0, -2]
    beta = data.iloc[0, -2] * 1000
    gamma1 = data.iloc[0, -4] * data.iloc[0, -5]*(10**6)
    gamma2 = data.iloc[0, -1] * data.iloc[0, -5]*(10**6)
    delta = gamma1 / gamma2
    theta = data.iloc[0, 4] * 1000
    mu = 0

def model(ini, time_step, params):
    Y = np.zeros(6) #column vector for the state variables
    S, E, I, A, R, D = ini
    alpha = params[0]
    beta = params[1]
    delta = params[2]
    gamma1 = params[3]
    gamma2 =params[4]
    theta = params[5]
    mu = 0

    Y[0] = - ( (beta*S*I + delta*A) / (S+E+I+A+R) ) - S + 100000 #S
    Y[1] = ( (beta*S*I + delta*A) / (S+E+I+A+R) ) - (mu +1)*E #E
    Y[2] = alpha*mu*E -(gamma1 + theta + 1)*I #I
    Y[3] = (1-alpha)*mu*E -(gamma + 1)*A #A
    Y[4] = gamma1*I + gamma2*A - R #R
    Y[5] = theta*I #D

    return Y

def x0fcn(data):
    data = data.iloc[0,:]
    E0 = data["tx_incid"]
    A0 = data["asymptomatiques"]
    I0 = data["tx_incid"] - data["asymptomatiques"]
    R0 = data["gueris"]
    D0 = data["deces"]
    S0 = 100000.0 - E0 - A0 - I0 - R0 - D0
    X0 = [S0, E0, A0, I0, R0, D0]

    return X0


def yfcn(res, params):
    return res[:,1]*params[2]

In [146]:
from scipy.stats import poisson
from scipy.stats import norm

from scipy.integrate import odeint as ode

def NLL(params, data, times): #negative log likelihood
    params = np.abs(params)
    # 	data = np.array(data)
    res = ode(model, sir_ode.x0fcn(params,data), times, args=(params,))
    y = yfcn(res, params)
    nll = sum(y) - sum(data*np.log(y))
    # note this is a slightly shortened version--there's an additive constant term missing but it 
    # makes calculation faster and won't alter the threshold. Alternatively, can do:
    # nll = -sum(np.log(poisson.pmf(np.round(data),np.round(y)))) # the round is b/c Poisson is for (integer) count data
    # this can also barf if data and y are too far apart because the dpois will be ~0, which makes the log angry

    # ML using normally distributed measurement error (least squares)
    # nll = -sum(np.log(norm.pdf(data,y,0.1*np.mean(data)))) # example WLS assuming sigma = 0.1*mean(data)
    # nll = sum((y - data)**2)  # alternatively can do OLS but note this will mess with the thresholds 
    #                             for the profile! This version of OLS is off by a scaling factor from
    #                             actual LL units.
    return nll

In [147]:
#minimum fischer matrix
def minifisher (times, params, data, delta = 0.001):
    #params = np.array(params)
    listX = []
    params_1 = np.array (params)
    params_2 = np.array (params)
    for i in range(len(params)):
        params_1[i] = params[i] * (1+delta)
        params_2[i]= params[i] * (1-delta)

        res_1 = ode(model, x0fcn(params_1,data), times, args=(params_1,))
        res_2 = ode(model, x0fcn(params_2,data), times, args=(params_2,))
        subX = (yfcn(res_1, params_1) - yfcn(res_2, params_2)) / (2 * delta * params[i])
        listX.append(subX.tolist())
    X = np.matrix(listX)
    FIM = np.dot(X, X.transpose())
    return FIM

In [150]:
def proflike (params, profindex, cost_func, times, data, perrange = 0.5, numpoints = 10):
    profrangedown = np.linspace(params[profindex], params[profindex] * (1 - perrange), numpoints).tolist()
    profrangeup = np.linspace(params[profindex], params[profindex] * (1 + perrange), numpoints).tolist()[1:] #skip the duplicated values
    profrange = [profrangedown, profrangeup]
    currvals = []
    currparams = []
    currflags = []

    def profcost (fit_params, profparam, profindex, data, times, cost_func):
        paramstest = fit_params.tolist()
        paramstest.insert(profindex, profparam)
        return cost_func (paramstest, data, times)

    fit_params = params.tolist() #make a copy of params so we won't change the origianl list
    fit_params.pop(profindex)
    print('Starting profile...')
    for i in range(len(profrange)):
        for j in profrange[i]:
            print(i, j)
            optimizer = optimize.minimize(profcost, fit_params, args=(j, profindex, data, times, cost_func), method='Nelder-Mead')
            fit_params = np.abs(optimizer.x).tolist() #save current fitted params as starting values for next round
            #print optimizer.fun
            currvals.append(optimizer.fun)
            currflags.append(optimizer.success)
            currparams.append(np.abs(optimizer.x).tolist())

    #structure the return output
    profrangedown.reverse()
    out_profparam = profrangedown+profrangeup
    temp_ind = range(len(profrangedown))
    temp_ind.reverse()
    out_params = [currparams[i] for i in temp_ind]+currparams[len(profrangedown):]
    out_fvals = [currvals[i] for i in temp_ind]+currvals[len(profrangedown):]
    out_flags = [currflags[i] for i in temp_ind]+currflags[len(profrangedown):]
    output = {'profparam': out_profparam, 'fitparam': np.array(out_params), 'fcnvals': out_fvals, 'convergence': out_flags}
    return output


In [ ]:
alpha = 
beta = final.iloc[0:-2]
delta = 
gamma1 = 
gamma2 =
theta = 
mu = 0